In [1]:
import pandas as pd
import numpy as np
import http.client
import requests
import json
from dotenv import load_dotenv
import os

In [2]:
# load environmental variables containing ZOOM tokens
load_dotenv('.env')

True

# Tokens and  Constants

In [3]:
# JWT token valid till 04/18/2022
token = os.getenv('ZOOM_JWT_TOKEN')

# cas main account user
user_id = os.getenv('ZOOM_USER_ID')

# general header using the jwt token
headers = {
    'authorization': "Bearer "+token,
    'content-type': "application/json"
}

# List User
This is where you get the user ID

In [6]:
conn = http.client.HTTPSConnection("api.zoom.us")
conn.request("GET", "/v2/users?status=active&page_size=30&page_number=1", headers=headers)

res = conn.getresponse()
data = res.read()

# print(data.decode("utf-8"))

# Webinars

## List webinar

In [7]:
url = 'https://api.zoom.us/v2/users/{userId}/webinars'.format(userId=user_id)
data = requests.get(
    url = url,
    headers =headers
)
data_json = data.json()
# print(json.dumps(data_json, indent=4))

## List Webinar Registrants

In [8]:
# List webinar registrants
webinar_id = 96528400763
url = 'https://api.zoom.us/v2/webinars/{webinarId}/registrants'.format(webinarId=webinar_id)
params = {
    'page_size':300, # max page size is 300,
}
# initiate a list of registrants
registrants = []

# if there is a next page token returned, this means that there is additional informaiton you need call using the next-page token. If there the next page token is an empty string, then you have retrieved all data
next_page_token = None

while next_page_token !='':

    # if this is not the initial run (where next_page_token is None), then add next_page_token parameter
    if next_page_token is not None:
        params['next_page_token'] = next_page_token

    data = requests.get(
        url = url,
        headers =headers,
        params=params
    )
    data_json = data.json()

    # retrieve next page token
    next_page_token = data_json['next_page_token']

    # add registrants to list
    registrants += (data_json['registrants'])

# double check that the length you retrieved is equal to total number of registrants
assert len(registrants) == data_json['total_records'], 'ERROR: The number of registrants you retrieved is ' + str(len(registrants)) + ', but the JSON returned ' + str(data_json['total_records']) +' records.'
# print(json.dumps(data_json, indent=4))

In [9]:
# conver to pandas dataframe
first_name = []
last_name = []
email = []

for record in registrants:
    try:
        first_name.append(record['first_name'])
    except:
        first_name.append(np.nan)
    try:
        last_name.append(record['last_name'])
    except:
        last_name.append(np.nan)
    try:
        email.append(record['email'])
    except:
        email.append(np.nan)

df = pd.DataFrame(
    {
        'first_name': first_name,
        'last_name': last_name,
        'email': email
    }
)

# check to see if email contains null value
print("Total registrants: ", df.shape[0])

# export records where email is not null
df_clean = df[pd.notna(df.email)]
df_clean.to_csv('cas-data/050821-adc-registrants.csv', index=False)

Total registrants:  575


## Register new webinar

In [12]:
first_name = 'Melanie'
last_name = 'Wu'
email_1 = 'myemail@gmail.com'
webinar_id = '99602079275' ## test webinar

url = 'https://api.zoom.us/v2/webinars/{webinarId}/registrants'.format(webinarId=webinar_id)

headers = {
    'authorization': "Bearer "+token,
    'content-type': "application/json"
}

data = {
    "email": email_1,
    "first_name": first_name,
    "last_name": last_name
}

#change dictionary into a json object/string
json_data = json.dumps(data)

response = requests.request("POST", url, data=json_data, headers=headers)

response.json()

{'registrant_id': 'H_aFXiKSRGaQ39nGh7aTUg',
 'id': 99602079275,
 'topic': 'test webinar - please keep this - I am using it to test new webiste',
 'start_time': '2021-07-23T03:00:00Z',
 'join_url': 'https://zoom.us/w/99602079275?tk=9g4wzLwpanGIzrbRTrBSODSmdTQ7HHxQxG8h1Fe7vPY.DQIAAAAXML8eKxZIX2FGWGlLU1JHYVEzOW5HaDdhVFVnAAAAAAAAAAAAAAAAAAAAAAAAAAAA&pwd=bXhPbTZtWXRYQUFtT3VnQXY2VEhXdz09&uuid=WN_BcXegKDJRrmAHII1xG6oGg'}

In [9]:
print(response.text)


{"registrant_id":"H_aFXiKSRGaQ39nGh7aTUg","id":99602079275,"topic":"test webinar - please keep this - I am using it to test new webiste","start_time":"2021-07-23T03:00:00Z","join_url":"https://zoom.us/w/99602079275?tk=9g4wzLwpanGIzrbRTrBSODSmdTQ7HHxQxG8h1Fe7vPY.DQIAAAAXML8eKxZIX2FGWGlLU1JHYVEzOW5HaDdhVFVnAAAAAAAAAAAAAAAAAAAAAAAAAAAA&pwd=bXhPbTZtWXRYQUFtT3VnQXY2VEhXdz09&uuid=WN_BcXegKDJRrmAHII1xG6oGg"}
